
---


The Business Case is 2-fold :

1. Segmentation: 

- Identify new user segmentation opportunities based on *__behavior data__* and your understanding of *__business challenges__* ;
- propose an approach to test the relevance of the segmentation 
- and how to implement it in the product


1. Plan To Improve
- Present a plan to improve (on your opinion) player matchmaking (= how Blitz decides who plays against whom),
- from proposing algorithm improvements to testing and implementing in the product


Instruction :

Deadline : 1 week

Format : 3 pages memo on Google Doc

---
---
# Segmenation tool - start
---

### Objective Assumption:
#### Our objective is to be able to segment users so that we can improve profit by seeing what behaviours relate paying users (payers) from non-paying users (non-payers). 


---

# Agenda:

Start Data Treatement
1. Load Data + quality check 
1. Exploratory Data Analysis (EDA)
1. Feature Engineering
1. Pre-processing

Start ML PIPELINE
1. Model Creation-Training
1. Evaluation
1. Results


---
---

# Imports

In [ ]:
# Main imports
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



# Secondary Imports
import matplotlib.ticker as ticker
import polars.selectors as cs

In [ ]:
#own functions
from visualizations import histo


In [ ]:
pl.Config.set_tbl_cols(200)  
pl.Config.set_tbl_rows(10)

> Given the size of data  Polars librerary will be a better suit instead of Pandas.

# ---- Start Data Treatement ----

# 1) Load Data

In [ ]:
df = pl.read_parquet("Copy of Copy of blitz_case_study.parquet")
df.head(3)


### Formating
Cast columns to the right format

In [ ]:
df = df.with_columns([
    pl.col('entry_price_dollars').cast(pl.Float64), 
])
df.head(3)

---
# 2)  EDA

In [ ]:
df.head(5)

In [ ]:
categorical_cols = ["user_id","game_id","match_type","entry_credit_type"]
numerical_cols = ["entry_price_dollars","entry_price_coins","prize_dollars","prize_coins","total_players","match_rank","seconds_until_result"]
temporal_cols = ["started_at"]
target = None

#not missing any col, assert should be true
assert len(categorical_cols + numerical_cols + temporal_cols) == len( df.columns), 'There are missing cols'

In [ ]:
#Quick look
for col in df.columns:
    unique_values = df.select(pl.col(col)).unique().to_series().to_list()
    unique_count = len(unique_values)
    print(col, "_"*(30-len(col)), " : unique count=> ", unique_count, " "*(7- int(len(str(unique_count)))),"| examples:", unique_values[:10])

### Duplicates

In [ ]:
df.filter(df.is_duplicated()).height

### Missing Values

In [ ]:
for col in df.columns:
  print(f"{col} = {df[col].null_count()}")

> No missing values

> No Duplicates

> The quality of the data seems to be ok.

### Adhoc Questions

In [ ]:
# Masks

dollar_type = df['entry_credit_type'] == "DOLLAR"
coin_type = df['entry_credit_type'] == "COIN"

coin_entry = ~df['entry_price_coins'].is_null() # coin_entry_nul
coin_outcome = ~df['prize_coins'].is_null() # coin_prix_nul

dollar_entry = ~df['entry_price_dollars'].is_null() # coin_entry_nul
dollar_outcome = ~df['prize_dollars'].is_null() # coin_prix_nul

In [ ]:
#Could there be a user  with NO coin entry and  prize coins?  -> YES,  dollar based
df.filter(~coin_entry &  ~coin_outcome ).head(2)
#df.filter(~coin_entry &  ~coin_outcome & coin_type).head(2) --> zero results, must have dollars


In [ ]:
#Could there be a user entry with NO Dollar entry and  Dollar Prize ?  -> Yes (coin based)
df.filter(~dollar_entry & dollar_outcome ).head(3)

In [ ]:
df.filter(~dollar_entry & dollar_outcome )['match_type'].unique()

> So the company can have a loss, in the exchange of coins for dollars

## Variables 

### Temporal

#### Started_at 

In [ ]:
df['started_at'].min(), df['started_at'].max(),

### Categorical

#### User_id

In [ ]:
print(f"unique users = {df['user_id'].n_unique()},  total rows = { df['user_id'].len()}")

In [ ]:
df['user_id'].n_unique()/df['user_id'].len() * 100

In [ ]:
af = df.group_by('user_id').len().rename({"len": "games_played"})
af2 = af.group_by('games_played').len().rename({"len":'users'}).sort(by='users', descending = True)
total_users = af2['users'].sum()
af2 = af2.with_columns(
    (pl.col("users") / total_users * 100).round(3).alias("%_total")
)
af2 = af2.with_columns(
    (pl.col("users").cum_sum() / total_users * 100).round(2).alias("cumul_percent")
)

af2.head(5)

In [ ]:
af2.select('cumul_percent').to_pandas().plot()

In [ ]:
af2.filter(af2['games_played']< 100).select('cumul_percent').to_pandas().plot()

> Almost 1/4th of users dont play more than 3 games and 50% dont play more than 15 games 

> --> This could be due to the data size, the last arrivals dont have time enought to play more.


#### Game_id

In [ ]:
df['game_id'].unique().len()

In [ ]:
df.group_by('game_id').len().to_pandas()

In [ ]:
game_counts = df.group_by('game_id').len().to_pandas()
histo(game_counts,'game_id', title='Number of records per games_id',symbol='M')

In [ ]:
game_counts = df.group_by(['game_id']).agg(pl.n_unique('user_id').alias('count')).to_pandas()
histo(game_counts,'game_id', title='Number of records per UNIQUE games_id',symbol='M')

#### Match_type 

In [ ]:
game_counts = df.group_by('match_type').len().to_pandas()
histo(game_counts,'match_type', title='Number of records per match_type',symbol='M')

> Clear preference

In [ ]:
game_counts = df.group_by(['game_id']).agg(pl.n_unique('match_type').alias('count')).to_pandas()
histo(game_counts,'game_id', title='Number of records per UNIQUE match_type',symbol='None' )

> Not all games have all match types

#### Entry_credit_type

In [ ]:
game_counts = df.group_by('entry_credit_type').len().to_pandas()
histo(game_counts,'entry_credit_type', title='Number of records per entry_credit_type',symbol='M')

In [ ]:
game_counts = df.group_by(['entry_credit_type']).agg(pl.n_unique('user_id').alias('count')).to_pandas()
histo(game_counts,'entry_credit_type', title='Number of records per UNIQUE entry_credit_type',symbol='M')

###  Numerical Variables

In [ ]:
df[numerical_cols].describe()

In [ ]:
for var in numerical_cols:
    print(var, "= ",df[var].n_unique())

#### Entry_price_dollars

In [ ]:
# get the most used entry prices in dollars

af = (df.filter(~df['entry_price_dollars'].is_null())
   .group_by('user_id')
   .agg(pl.col('entry_price_dollars').mode().alias('mode_prices'))
   .explode('mode_prices')
   .rename({'mode_prices': 'entry_price_dollars'}))
af.head(9)

In [ ]:
game_counts = af.group_by('entry_price_dollars').len().to_pandas()
histo(game_counts,'entry_price_dollars', title='Most used entry_price_dollars',symbol='D')

In [ ]:
game_counts = af.group_by('entry_price_dollars').len().to_pandas()
game_counts = game_counts[game_counts.entry_price_dollars < 10]
histo(game_counts,'entry_price_dollars', title='Usual entry price in dollars (ZOOMED)',symbol='D')

> The usual betting for majority of users is less than 3 dolars per game.

#### Entry_price_coins

In [ ]:
af = (df.filter(~df['entry_price_coins'].is_null())
   .group_by('user_id')
   .agg(pl.col('entry_price_coins').mode().alias('mode_prices'))
   .explode('mode_prices')
   .rename({'mode_prices': 'entry_price_coins'}))
af.head(9)


In [ ]:
game_counts = af.group_by('entry_price_coins').len().to_pandas()
histo(game_counts,'entry_price_coins', title='Most used entry_price_coins',symbol='D')

#### Prize_dollars 

In [ ]:
df.filter(df['prize_dollars']>0).head(4)

In [ ]:
df.filter(df['prize_dollars']>0)['prize_dollars'].to_pandas().hist(bins=30)


In [ ]:
af = (df.filter(df['prize_dollars'] > 0)
   .group_by('user_id')
   .agg(pl.col('prize_dollars').mode().alias('mode_prices'))
   .explode('mode_prices')
   .rename({'mode_prices': 'prize_dollars'}))
af.head(9)

game_counts = af.group_by('prize_dollars').len().to_pandas()
histo(game_counts,'prize_dollars', title='Most used prize_dollars',symbol='D')


#### Prize_coins

In [ ]:
df.filter(df['prize_coins']>0)['prize_coins'].to_pandas().hist(bins=10)


In [ ]:
af = (df.filter(df['prize_coins'] > 0)
   .group_by('user_id')
   .agg(pl.col('prize_coins').mode().alias('mode_prices'))
   .explode('mode_prices')
   .rename({'mode_prices': 'prize_coins'}))
af.head(9)

game_counts = af.group_by('prize_coins').len().to_pandas()
histo(game_counts,'prize_coins', title='Most used prize_coins',symbol='D')


#### Total_players

In [ ]:
df.filter(df['total_players']>0)['total_players'].to_pandas().hist(bins=10)


---
## Adhoc Questions

### 1. When does a user start putting real money? (first dollar entry_credit_type)

In [ ]:
#order
sorted_df = df.sort(["user_id", "started_at"], descending=[False, False])
sorted_df = sorted_df.with_columns(
    pl.when(pl.col("seconds_until_result") > 0)
      .then(1)
      .otherwise(0)
      .alias("is_completed"),
)


In [ ]:
sorted_df = sorted_df.with_columns(
   pl.col('is_completed')
      .cum_sum()
      .over(['user_id'])
      .alias("game_number")
 )

sorted_df = sorted_df.with_columns(
    pl.when((pl.col("is_completed") == 1) & (pl.col("entry_credit_type") == "DOLLAR"))
    .then(1)
    .otherwise(0)
    .alias("completed_dollar_game")
)

first_dollar_games = (
    sorted_df
    .filter(pl.col("completed_dollar_game") == 1)
    .group_by("user_id")
    .agg(pl.min("game_number").alias("first_dollar_game_number"))
)
 
sorted_df = sorted_df.join(
    first_dollar_games,
    on="user_id",
    how="left"
)

# adding day difference
sorted_df = sorted_df.with_columns(
    pl.col("started_at") 
    .min()
    .over("user_id")
    .alias("first_game_date")
)


sorted_df = sorted_df.with_columns(
    (pl.col("started_at") - pl.col("first_game_date"))
    .dt.total_days()
    .floor()
    .cast(pl.Int64).alias("days_since_first_game")
)

  

In [ ]:
# proof first_dollar_game_number works well
# user = 'JbAAfDVPSIdl0Xjr5lCk6V7A7nr1'
# sorted_df.filter(sorted_df['user_id'] == user).tail(30)

In [ ]:
# How many days per user until played a DOLLAR entry credit type game
first_dollar_game_number_per_user = sorted_df.group_by('user_id')\
    .agg(pl.col('first_dollar_game_number')
    .min())
first_dollar_game_number_per_user.head(3)


In [ ]:
#show only non null users with less than 20 days
first_dollar_game_number_per_user.filter(\
  (~first_dollar_game_number_per_user['first_dollar_game_number'].is_null())\
    & (first_dollar_game_number_per_user['first_dollar_game_number'] <20))\
      .to_pandas().hist(bins=300)

In [ ]:
#show only non null users with cumulatative curb

filtered = first_dollar_game_number_per_user.filter(
    (~first_dollar_game_number_per_user['first_dollar_game_number'].is_null()) 

).to_pandas()


filtered['first_dollar_game_number'].hist(
    bins=300, 
    cumulative=True, 
    density=True,
    histtype='step'
)
plt.xlabel('First Dollar Game Number')
plt.ylabel('Cumulative Proportion')
plt.title('Cumulative Distribution of First Dollar Game Number')
plt.show()

In [ ]:
# Make a zoom to 100 max games
filtered = first_dollar_game_number_per_user.filter(
    (~first_dollar_game_number_per_user['first_dollar_game_number'].is_null()) 

).to_pandas()


filtered['first_dollar_game_number'].hist(
    bins=300, 
    cumulative=True, 
    density=True,
    histtype='step'
)
plt.xlabel('First Dollar Game Number')
plt.ylabel('Cumulative Proportion')
plt.title('Cumulative Distribution of First Dollar Game Number')
# For Zoom
plt.xlim(0, 100)


plt.show()

> If the users are going to become dollar gamblers, 40 games should be enough for the majority.

### 2. How many days does this represents on avg? 

In [ ]:
# Get the day when they started playing with Dollars since first day played.
days_since_first_game_per_user = sorted_df\
    .filter(sorted_df['game_number'] == sorted_df['first_dollar_game_number'])\
        .group_by('user_id')\
            .agg(pl.col('days_since_first_game')\
                .min())
aux = days_since_first_game_per_user.to_pandas()

aux['days_since_first_game'].hist(
    bins=300, 
    cumulative=True, 
    density=True,
    histtype='step'
)
plt.xlabel('Days to play a Dollar entry game')
plt.ylabel('Cumulative Proportion')
plt.title('Cumulative Distribution of days_since_first_game')
plt.show()


> By day 5 most users will already be playing a Dollar game, and by day 20 almost everyone (this is due to our constrains in data, check next result).

> We will use these features (first_dollar_game_number_per_user and days_since_first_game_per_user ) in Feature Engineering.

### 3. What is the seniority in our data?

In [ ]:
#Label Seniority (days_since_first_game)
min_start_date =  df.group_by(['user_id']).agg(pl.col('started_at').min().alias('min_start_at'))
af = df.join(min_start_date, on='user_id', how='left')
af = af.with_columns((pl.col("started_at") - pl.col("min_start_at")).dt.total_days().alias("days_since_first_game"))

af.group_by('user_id').agg(pl.col('days_since_first_game').max()).to_pandas().hist(bins=30)



In [ ]:
df['started_at'].min(), df['started_at'].max()

---
# 3) Feature Engineering


**Objective:**\
The segmentation aims to study users monetization behavior. \

Meaning: we want to know the __**behaviours**__ that relates users that use DOLLARS ('PAYERS') \
and the opposite behavior, the NON PAYERS. This will give insights on the actions that can take for converting Non-Payers to Payers.

**TO DO:**
1) Mark players who had and had NOT betted at least one game (Label: payers / non-payers  )
1) Mark players who probably didnt have enough time to bet (30 days) and take them out of the study (Label: 30Days_to_play).
1) Create Dummies, for later usage.
1) Summarize per user, in one row, as much knowledge on behaviour that we can from the data we posses (feature engineering).
1) Convert to pandas for pre-processing.

**Expected outcome:**  features_df


In [ ]:
df.head(3)

#### a) Label: payers / non-payers classification

In [ ]:
is_payer_table = df.join(first_dollar_game_number_per_user, on="user_id", how="left")

is_payer_table = is_payer_table.with_columns(
    pl.when(pl.col("first_dollar_game_number")>0)
    .then(1)
    .otherwise(0)
    .alias("is_payer")
)

is_payer_table = (
    is_payer_table.group_by("user_id")\
        .agg( pl.col("is_payer").max().alias("is_payer"))
    )

# Ensure no duplicates
assert is_payer_table.filter(is_payer_table['user_id'].is_duplicated()).shape[0] == 0

is_payer_table.head(2)

#### b)  Feature : 30Days_to_play

In [ ]:
# BOOL Markers

#Player had enough days to become payer
DataBase_last_day = df['started_at'].max()

valid_users_30days = df.with_columns(
    pl.when(pl.col("started_at") < DataBase_last_day - pl.duration(days=30))
    .then(1)
    .otherwise(0)
    .alias("had_30Days_to_play")
)


valid_users_30days = valid_users_30days.with_columns(
    pl.col("had_30Days_to_play")\
        .max()\
        .over("user_id").alias("had_30Days_to_play")
)

valid_users_30days.head(2)

#### c) Dummies variables

We want to have a table for each user where we mark if it has ever been in a specific game_id, match_type and entry_credit_type.

Columns : "game_id","match_type","entry_credit_type"

**Output:** dummies_summarized (table)

In [ ]:

#Select dummy cols
cols_to_dummy = ["game_id","match_type","entry_credit_type"]

# Create all dummies
all_dummies = df.to_dummies(cs.by_name(cols_to_dummy))

#list of cols to keep
cols_to_keep = ['user_id']
for col in cols_to_dummy:
  cols_to_keep = cols_to_keep +[ col+"_"+dummy_col for dummy_col in df[col].unique().to_list()]


#keep only necessary
all_dummies = all_dummies[cols_to_keep]

# Ensure all dummies are present
assert len( df['game_id'].unique().to_list() + df['match_type'].unique().to_list() + df['entry_credit_type'].unique().to_list() + ['user_id']) == len(all_dummies.columns)


#create base table for summarizing dummies (this will)
dummies_summarized = all_dummies.select(["user_id"]).unique()

for col in list(set(cols_to_keep) - set(['user_id'])):
  result = all_dummies.group_by("user_id")\
    .agg(\
       pl.col(col).max().alias("has_done_"+col)
       )
  dummies_summarized = dummies_summarized.join(result, on="user_id", how="left")


# Ensure no duplicates
assert dummies_summarized.filter(dummies_summarized['user_id'].is_duplicated()).shape[0] == 0

dummies_summarized.head(2)

In [ ]:
# ## Test  >>> OK 
# m1 = df['user_id']== 'y12WV4WQ0NeIAP7uCV4ubYrGpI42'

# df.filter(m1)

#### d) Seniority (days_since_first_game) 
This could be used as filter

In [ ]:
#Label Seniority (days_since_first_game)
min_start_date =  df.group_by(['user_id']).agg(pl.col('started_at').min().alias('min_start_at'))
base_df = df.join(min_start_date, on='user_id', how='left')
base_df = base_df.with_columns((pl.col("started_at") - pl.col("min_start_at")).dt.total_days().alias("days_since_first_game"))

base_df.tail(2)

#### e) User Behavior Features
 Here we add all the BEHAVIORAL features we can use.

In [ ]:
# Label games completed

base_df = base_df.with_columns(
    pl.when(pl.col("seconds_until_result") > 0)
      .then(1)
      .otherwise(0)
      .alias("game_is_completed"))
base_df.head(2)



In [ ]:
# base_df = base_df.filter(base_df['days_since_first_game'] < 10)

user_behavior_features = (
    base_df.group_by("user_id").agg(
        
        # --- Seniority ---
        pl.col("days_since_first_game").max().alias("seniority_in_days"),

        # --- Game Usage ---
        pl.col("game_is_completed").sum().alias("total_played_games"),
        pl.col("seconds_until_result").mean().alias("avg_game_duration"),
        pl.col("seconds_until_result").std().alias("game_duration_std"),
        pl.col("game_id").mode().first().alias("most_played_game_id"),
        pl.col("match_type").mode().first().alias("most_played_match_type"),
        # --- Spending Metrics ---
        ## Dollars
        pl.col("entry_price_dollars").mean().alias("avg_entry_dollars"), # <-- Remake
        pl.col("entry_price_dollars").sum().alias("total_spend_dollars"),
        pl.col("entry_price_dollars").std().alias("spend_dollars_std"), # <--- Remake
        pl.col("prize_dollars").sum().alias("total_prize_dollars"),
        
        ## Coins
        pl.col("entry_price_coins").mean().alias("avg_entry_coins"),
        pl.col("entry_price_coins").sum().alias("total_spend_coins"),
        pl.col("entry_price_coins").std().alias("spend_coins_std"),
        pl.col("prize_coins").sum().alias("total_prize_coins"),

        # --- Game Outcome ---
        pl.len().alias("match_count"),
        pl.col("match_rank").mean().alias("avg_rank"),
        pl.col("match_rank").min().alias("best_rank"),
        pl.col("match_rank").max().alias("worst_rank"),

        # --- Time Patterns ---
        pl.col("started_at").dt.hour().mode().first().alias("most_played_hour"),
        (pl.col("started_at").dt.hour().is_between(0, 5)).mean().alias("night_matches"),
        (pl.col("started_at").dt.hour().is_between(6, 11)).mean().alias("morning_matches"),
        (pl.col("started_at").dt.hour().is_between(12, 17)).mean().alias("afternoon_matches"),
        (pl.col("started_at").dt.hour().is_between(18, 23)).mean().alias("evening_matches"),
        pl.col("started_at").dt.weekday().mode().first().alias("most_played_weekday"),
        (pl.col("started_at").dt.weekday().is_in([0, 1, 2, 3, 4])).mean().alias("weekday_user"),
        (pl.col("started_at").dt.weekday().is_in([5, 6])).mean().alias("weekend_user"),

        # --- Player Metrics ---
        pl.col("total_players").mean().alias("avg_players_per_match"),

        # --- Win Metrics ---
        (pl.col("match_rank") == 1).sum().alias("first_place_count"),
        (pl.col("match_rank") <= 3).sum().alias("top_three_count"),
        (pl.col("match_rank") > 3).sum().alias("loss_count"),  # Can be removed if unnecessary

        # --- Game Variety Exposure ---
        pl.col("match_type").n_unique().alias("match_type_variety_exposure"),

        # --- Paid Matches ---
        pl.col("entry_price_dollars").filter(pl.col("entry_price_dollars") > 0).count().alias("paid_match_count"),
    )
)

user_behavior_features.head(2)


In [ ]:
base_df = df.filter(~df['entry_price_dollars'].is_null())
avg_entry_dollars = (
    base_df
    .group_by("user_id")
    .agg(pl.col("entry_price_dollars").mean().alias("avg_entry_price"))
)

avg_entry_dollars

#### e) Composed features (metrics)

In [ ]:
user_behavior_features = user_behavior_features.with_columns([
    # Net profit/loss
    (pl.col("total_prize_dollars") - pl.col("total_spend_dollars")).alias("net_dollar_profit"),
    (pl.col("total_prize_coins") - pl.col("total_spend_coins")).alias("net_coin_profit"),
    
    # Performance metrics
    (pl.col("first_place_count") / pl.col("match_count")).alias("win_rate"),
    (pl.col("top_three_count") / pl.col("match_count")).alias("top_three_rate"),
    
    # Monetization behavior
    (pl.col("paid_match_count") / pl.col("match_count")).alias("paid_game_ratio"),
    
    #win_rate categories (low, medium, high)
    #total spend ratio (dollars/coins)

])

user_behavior_features.head(2)

In [ ]:
cols_to_dummy =["most_played_game_id","most_played_match_type"]

# Create all dummies
usr_behavior_dummies = user_behavior_features.to_dummies(cs.by_name(cols_to_dummy))

#list of cols to keep
cols_to_keep = ['user_id']
for col in cols_to_dummy:
  cols_to_keep = cols_to_keep +[ col+"_"+dummy_col for dummy_col in user_behavior_features[col].unique().to_list()]

#keep only necessary
usr_behavior_dummies = usr_behavior_dummies[cols_to_keep]

# Ensure all dummies are present
assert len(  user_behavior_features['most_played_game_id'].unique().to_list() +\
  user_behavior_features['most_played_match_type'].unique().to_list() +\
    ['user_id']) == len(usr_behavior_dummies.columns)


#create base table for summarizing dummies (this will)
usr_dummies_summarized = usr_behavior_dummies.select(["user_id"]).unique()

for col in list(set(cols_to_keep) - set(['user_id'])):
  result = usr_behavior_dummies.group_by("user_id")\
    .agg(\
       pl.col(col).max().alias(col)
       )
  usr_dummies_summarized = usr_dummies_summarized.join(result, on="user_id", how="left")


# Ensure no duplicates
assert usr_dummies_summarized.filter(usr_dummies_summarized['user_id'].is_duplicated()).shape[0] == 0

usr_dummies_summarized.head(2)

### f) Merge Data Frames 
user_features + user_behavior_w_dummies + is_payer_table + valid_users_30days 

In [ ]:
first_dollar_game_number_per_user.head(2), days_since_first_game_per_user.head(2)

In [ ]:
# Merge with dummies

features_df_polars = user_behavior_features.join(usr_dummies_summarized, on='user_id', how='left') 
features_df_polars = features_df_polars.drop(cols_to_dummy)

# features_df_polars = features_df_polars.join(is_payer_table, on="user_id", how="left") # this depends on given dates filter
# features_df_polars = features_df_polars.join(valid_users_30days, on="user_id", how="left") # seniority will be enough


# This tells at which game did the player used DOLLAR as entry_credit_type (NULL if has never played with dollar)
features_df_polars = features_df_polars.join(first_dollar_game_number_per_user, on="user_id", how="left")

#Tells days since first game until Dollar as entry_credit_type game.
features_df_polars = features_df_polars.join(days_since_first_game_per_user, on="user_id", how="left")

In [ ]:
features_df_polars.columns

## g) Check output

In [ ]:
# Quick look with null value check - simplified version
cols_with_nulls = []
for col in features_df_polars.columns:
    # Count nulls directly
    null_count = features_df_polars.select(pl.col(col).is_null().sum()).item()
    has_nulls = "Yes" if null_count > 0 else "No"
    cols_with_nulls.append(col) if null_count > 0 else None
    
    # Show a few sample values without counting unique values
    try:
        sample_values = features_df_polars.select(pl.col(col)).head(5).to_series().to_list()
    except:
        sample_values = ["Error getting samples"]
    
    print(
        f"{col} {'_' * (30 - len(col))} : nulls=> {has_nulls} ({null_count}) | "
        f"sample values: {sample_values}"
    )
print(f"Columns with missing values: {cols_with_nulls}")

In [ ]:
features_df = features_df_polars.to_pandas()

In [ ]:
features_df.columns.to_list()

In [ ]:
[x for x in features_df.columns.to_list() if x.__contains__("most_played")]

In [ ]:
features_df.info()

In [ ]:
drop_col_features = [
  # irrelevant
  'user_id',
  # drop first element in category
  # 'afternoon_matches',
  # 'weekday_user',
  # 'most_played_game_id_Golf',
  # 'most_played_match_type_ONE_VS_ONE'  
  ]

features_df_dropped = features_df.drop(columns=drop_col_features)
features_df.head(2)

### Missing Values

In [ ]:
plt.figure(figsize=(10, 4))
sns.heatmap(features_df_dropped.isnull(), cbar=False, cmap='viridis')


In [ ]:
missing_val_cols = features_df_dropped.columns[features_df_dropped.isnull().any()].to_list()
missing_val_cols

In [ ]:
for col in missing_val_cols:
  print(col, " => min = ",features_df_dropped[col].min())

In [ ]:
fill_value = {
  'game_duration_std': 0,
 'avg_entry_dollars': 0,
 'spend_dollars_std': 0,
 'avg_entry_coins': 0,
 'spend_coins_std': 0,
 'first_dollar_game_number': 10000,
 'days_since_first_game': 10000 
}

In [ ]:
# will fill in preprocessing
#features_df_dropped_filled = features_df_dropped.fillna(value=fill_value)

In [ ]:
# Check Point : save to parquet
features_df_dropped.to_parquet("X.parquet")

# ---- Start ML PIPELINE ----

In [1]:
# Base libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Pre processing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Models
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors

# pipeline
import optuna
from sklearn.model_selection import cross_val_score, LeaveOneOut, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV 

# Metrics, viz
from utils import dbscan_scorer # (custome score)
import umap
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


In [2]:
# Load
df = pd.read_parquet("X.parquet")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158009 entries, 0 to 158008
Data columns (total 57 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   seniority_in_days                      158009 non-null  int64  
 1   total_played_games                     158009 non-null  int32  
 2   avg_game_duration                      158009 non-null  float64
 3   game_duration_std                      139355 non-null  float64
 4   avg_entry_dollars                      94782 non-null   float64
 5   total_spend_dollars                    158009 non-null  float64
 6   spend_dollars_std                      82954 non-null   float64
 7   total_prize_dollars                    158009 non-null  float64
 8   avg_entry_coins                        153354 non-null  float64
 9   total_spend_coins                      158009 non-null  int64  
 10  spend_coins_std                        131688 non-null  

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
seniority_in_days,158009.0,11.158997,11.839562,0.0,0.000000,6.000000,24.000000,30.000000
total_played_games,158009.0,95.393592,227.733900,1.0,4.000000,15.000000,78.000000,5679.000000
avg_game_duration,158009.0,1033.761713,2142.486921,7.0,358.600000,576.657895,969.722751,105280.000000
game_duration_std,139355.0,1787.240898,3447.653322,0.0,442.992820,956.062251,1638.591637,120689.692520
avg_entry_dollars,94782.0,1.546178,2.034863,0.2,0.814815,1.000000,1.576000,95.163636
total_spend_dollars,158009.0,77.441952,934.115158,0.0,0.000000,1.800000,18.200000,141200.000000
spend_dollars_std,82954.0,0.982262,1.702943,0.0,0.200000,0.467862,1.176856,60.864580
total_prize_dollars,158009.0,58.766828,796.471791,0.0,0.000000,0.500000,10.200000,134958.800000
avg_entry_coins,153354.0,256.284003,287.806726,15.0,60.215054,122.562539,300.000000,1000.000000
total_spend_coins,158009.0,6824.199634,12863.029315,0.0,1000.000000,2050.000000,7450.000000,490050.000000


# 1) Pre-Processing

In [5]:
# Features by category

numeric_features = [
    'seniority_in_days', 'total_played_games', 'avg_game_duration', 'game_duration_std',
    'avg_entry_dollars', 'total_spend_dollars', 'spend_dollars_std', 'total_prize_dollars',
    'avg_entry_coins', 'total_spend_coins', 'spend_coins_std', 'total_prize_coins',
    'match_count', 'avg_rank', 'best_rank', 'worst_rank', 'most_played_hour',
    'night_matches', 'morning_matches', 'afternoon_matches', 'evening_matches',
    'most_played_weekday', 'weekday_user', 'weekend_user', 'avg_players_per_match',
    'first_place_count', 'top_three_count', 'loss_count', 'match_type_variety_exposure',
    'paid_match_count', 'net_dollar_profit', 'net_coin_profit', 'win_rate',
    'top_three_rate', 'paid_game_ratio'
]

categorical_features = [
    'most_played_game_id_WheelOfFortune', 'most_played_game_id_Pool',
    'most_played_game_id_PokerBlast', 'most_played_match_type_SPIN_AND_GO',
    'most_played_game_id_21Blitz', 'most_played_game_id_Yatzy',
    'most_played_game_id_Domino', 'most_played_game_id_BubbleShooter',
    'most_played_game_id_Match3', 'most_played_game_id_Golf',
    'most_played_game_id_GridBlocks', 'most_played_game_id_Bingo',
    'most_played_game_id_TripeaksSolitaire', 'most_played_match_type_ONE_VS_ONE',
    'most_played_game_id_KlondikeSolitaire', 'most_played_match_type_BRAWL',
    'most_played_game_id_FindObjects', 'most_played_game_id_TileMatch',
    'most_played_game_id_GinRummy', 'most_played_game_id_BallBlast'
]

# Numeric 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Median for skewness (by default is mean otherwise)
    ('scaler', StandardScaler())                   
])

# Categorical imputer (use most frequent value)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing cat
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # Dummies
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

#check no col is missing
assert len([x for x in numeric_features+categorical_features if x not in df.columns.tolist()]) == 0

## 1) Pipeline

In [6]:
# Pipelines

dbscan_pipeline = Pipeline([('preprocessor', preprocessor), ('dbscan', DBSCAN())])
kmeans_pipeline = Pipeline([('preprocessor', preprocessor), ('kmeans', KMeans())])
hierarchical_pipeline = Pipeline([('preprocessor', preprocessor), ('hierarchical', AgglomerativeClustering())])

# 2) Domain and Model definition

In [7]:
#Using  RandomizedSearchCV

# #NOTE: we are not interested in clustering 2 (paid vs not paid most likely)
# dbscan_params = {'dbscan__eps': [0.1, 0.3, 0.5, 0.7, 1.0], 'dbscan__min_samples': [5, 10, 15, 20]}
# kmeans_params = {'kmeans__n_clusters': [3, 4, 5]}
# hierarchical_params = {'hierarchical__n_clusters': [ 3, 4, 5]} 


# Using Optuna

# Parameter Space

def dbscan_param_space(trial):
    return {
        'dbscan__eps': trial.suggest_float('dbscan__eps', 0.1, 10.0),
        'dbscan__min_samples': trial.suggest_int('dbscan__min_samples', 2, 20)
    }

def kmeans_param_space(trial):
    return {
        'kmeans__n_clusters': trial.suggest_int('kmeans__n_clusters', 3, 10),
        'kmeans__init': trial.suggest_categorical('kmeans__init', ['k-means++', 'random']),
        'kmeans__max_iter': trial.suggest_int('kmeans__max_iter', 100, 500)
    }

def hierarchical_param_space(trial):
    return {
        'hierarchical__n_clusters': trial.suggest_int('hierarchical__n_clusters', 3, 10),
        'hierarchical__affinity': trial.suggest_categorical('hierarchical__affinity', ['euclidean', 'l1', 'l2']),
        'hierarchical__linkage': trial.suggest_categorical('hierarchical__linkage', ['ward', 'complete', 'average'])
    }
    
    
# Objective functions
def dbscan_objective(trial, X, y=None):
    params = dbscan_param_space(trial)
    dbscan_pipeline.set_params(**params)
    
    try:
        X_processed = preprocessor.fit_transform(X)
        dbscan = dbscan_pipeline.named_steps['dbscan']
        dbscan.set_params(**{k.replace('dbscan__', ''): v for k, v in params.items()})
        labels = dbscan_pipeline.fit_predict(X)
        # Base on custom scorer (from utils.py)
        score = dbscan_scorer(dbscan_pipeline, X, labels)
        return score
    except Exception as e:
            print(f"Error in DBSCAN trial: {e}")
            return -1.0

def kmeans_objective(trial, X, y=None):
    params = kmeans_param_space(trial)
    kmeans_pipeline.set_params(**params)
    
    try:
        X_processed = preprocessor.fit_transform(X)
        kmeans = kmeans_pipeline.named_steps['kmeans']
        kmeans.set_params(**{k.replace('kmeans__', ''): v for k, v in params.items()})
        # Base on Silhouette Score
        labels = kmeans_pipeline.fit_predict(X)
        score = silhouette_score(X, labels)
        return score
    except Exception as e:
            print(f"Error in KMeans trial: {e}")
            return -1.0

def hierarchical_objective(trial, X, y=None):
    params = hierarchical_param_space(trial)
    hierarchical_pipeline.set_params(**params)
    
    try:
        X_processed = preprocessor.fit_transform(X)
        hierarchical = hierarchical_pipeline.named_steps['hierarchical']
        hierarchical.set_params(**{k.replace('hierarchical__', ''): v for k, v in params.items()})
        # Base on Silhouette Score
        labels = hierarchical_pipeline.fit_predict(X)
        score = silhouette_score(X, labels)
        return score
    except Exception as e:
            print(f"Error in Hierarchical trial: {e}")
            return -1.0




In [8]:
# Models instantiation

# # Using Random Search
# dbscan_random_search = RandomizedSearchCV(
#     dbscan_pipeline,
#     dbscan_params,
#     scoring=dbscan_scorer,
#     cv=5,
#     n_iter=10,
#     random_state=42
# )

# # For KMeans and Hierarchical Clustering, use default scoring (Silhouette Score)
# kmeans_random_search = RandomizedSearchCV(
#     kmeans_pipeline,
#     kmeans_params,
#     scoring='silhouette_score',
#     cv=5,
#     n_iter=10,
#     random_state=42
# )

# hierarchical_random_search = RandomizedSearchCV(
#     hierarchical_pipeline,
#     hierarchical_params,
#     scoring='silhouette_score',
#     cv=5,
#     n_iter=10,
#     random_state=42
# )


# 3) Fitting models

> Please note that each model has its own pipeline and scaling is done internally

In [9]:
## Useing Random Search
# dbscan_random_search.fit(df)
# kmeans_random_search.fit(df)
# hierarchical_random_search.fit(df)

In [10]:
## Using OPTUNA

X = df.copy()

# Optimize using Optuna
def optimize_clustering(objective_func, X, n_trials=50):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective_func(trial, X), n_trials=n_trials)
    return study.best_params, study.best_value


In [11]:
# Sample Fitting

In [12]:
# Using SAMPLE
sample_size = 50000  
X_sample = X.sample(min(sample_size, len(X)), random_state=42)


In [13]:
# Using DBSCAN
print("Optimizing DBSCAN...")
best_dbscan_params, best_dbscan_score, best_dbscan_trial = optimize_clustering(dbscan_objective, X_sample, n_trials=20)
print(f"Best DBSCAN Params: {best_dbscan_params}, Best Score: {best_dbscan_score}")

# Using KMeans
print("Optimizing KMeans...")
best_kmeans_params, best_kmeans_score, best_kmeans_trial = optimize_clustering(kmeans_objective, X_sample, n_trials=20)
print(f"Best KMeans Params: {best_kmeans_params}, Best Score: {best_kmeans_score}")

# Using Hierarchical Clustering
print("Optimizing Hierarchical Clustering...")
best_hierarchical_params, best_hierarchical_score, best_hierarchical_trial = optimize_clustering(hierarchical_objective, X_sample, n_trials=20)
print(f"Best Hierarchical Params: {best_hierarchical_params}, Best Score: {best_hierarchical_score}")


print("\n --------Overall best model---------")
best_scores = {
    'DBSCAN': best_dbscan_score,
    'KMeans': best_kmeans_score,
    'Hierarchical': best_hierarchical_score
}

best_model_name = max(best_scores, key=best_scores.get)
print(f"Best overall model: {best_model_name} with score {best_scores[best_model_name]}")


[I 2025-03-28 14:19:21,749] A new study created in memory with name: no-name-4883f6a0-aa9a-415d-9009-c133ac739e59


Optimizing DBSCAN...


[I 2025-03-28 14:23:23,691] Trial 0 finished with value: -1.0 and parameters: {'dbscan__eps': 8.42938948787148, 'dbscan__min_samples': 14}. Best is trial 0 with value: -1.0.


In [ ]:
# Apply Best Model

In [ ]:
X_processed = preprocessor.fit_transform(X)

In [ ]:
# Select best model
if best_model_name == 'DBSCAN':
    best_model = DBSCAN(**{k.replace('dbscan__', ''): v for k, v in best_dbscan_params.items()})
elif best_model_name == 'KMeans':
    best_model = KMeans(**{k.replace('kmeans__', ''): v for k, v in best_kmeans_params.items()})
else: 
    best_model = AgglomerativeClustering(**{k.replace('hierarchical__', ''): v for k, v in best_hierarchical_params.items()})


In [ ]:
# Post Analysis

In [ ]:
print("\nCluster distribution:")
print(X['cluster'].value_counts())

In [ ]:
# Check Analysis

# TODO:


A matchmaking system needs to:

1. Represent users behaviors as structured data
1. Find suitable matches based on similarity or compatibility rules
1. Have a ranking system for potential matches according to relevance (previous point)
1. Be Scalable, meaning it must be able to handle many users and frequent updates. 

Possible systems:


1)  vector embeded system: Vector databases are high-dimensional vectors (numerical representations) and allow for similarity searches, they rely on space as data is translated to vectors.

Problems:
- Balancing factors: This is hard part as you need to weight space.
- Interpretability: They are less straight foward on how the decision was taken.
- Cold start problem: With no data, creating a meaningful vectors initially would need a strategy to handle this.

---
---
---
# BROULILLON

In [ ]:

df.filter(df['user_id'] == 'Xbkcq0zL9hbwILnFpTSfO5Zd9jm2').sort(by=['started_at'])

In [ ]:
user = 'Xbkcq0zL9hbwILnFpTSfO5Zd9jm2'
df.filter(df['user_id'] == user) ['started_at'].min(),df.filter(df['user_id'] == user) ['started_at'].max()